![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [2]:
// We need to load assemblies at the start in their own cell
#load "../Initialize.csx"
#load "../QuantConnect.csx"

using System;
using System.Linq;
using System.Collections.Generic;

using QuantConnect;
using QuantConnect.Data.Market;
using QuantConnect.Research;

In [3]:
using System;
using System.Linq;
using System.Collections.Generic;
using Newtonsoft.Json;

// =====================================
// Helpers
// =====================================

decimal Round2(decimal value) => Math.Round(value, 2);

decimal Median(List<decimal> values)
{
    if (values == null || values.Count == 0) return 0m;
    var sorted = values.OrderBy(x => x).ToList();
    int n = sorted.Count;
    int mid = n / 2;
    return (n % 2 == 0)
        ? (sorted[mid - 1] + sorted[mid]) / 2m
        : sorted[mid];
}

decimal StdDev(List<decimal> values)
{
    if (values == null || values.Count == 0) return 0m;
    var mean = values.Average();
    var sumSq = values.Sum(v => (v - mean) * (v - mean));
    return (decimal)Math.Sqrt((double)(sumSq / values.Count));
}

// =====================================
// DTO for JSON output (camelCase)
// =====================================

public class WeeklyBarRecord
{
    public string startDate { get; set; }
    public string endDate { get; set; }
    public decimal open { get; set; }
    public decimal high { get; set; }
    public decimal low { get; set; }
    public decimal close { get; set; }
    public decimal volume { get; set; }
    public int tradingDays { get; set; }
    public decimal openToHigh { get; set; }
    public decimal openToLow { get; set; }
    public decimal openToClose { get; set; }
    public decimal totalDelta { get; set; }
}

In [4]:
// =====================================
// QuantBook setup
// =====================================

var qb = new QuantBook();

const int MinTradingDays = 4;
const string TickerSymbol = "GLD";
const int TotalDeltaDailyThreshold = 10;

var HistoryStartDate = new DateTime(2004, 11, 22);
var HistoryEndDate   = new DateTime(2025, 12, 1);

// Add equity
var equity = qb.AddEquity(TickerSymbol, Resolution.Daily);

// Use SplitAdjusted normalization (will still be cleaned by outlier filters)
equity.SetDataNormalizationMode(DataNormalizationMode.SplitAdjusted);

// Load daily bars
var allDailyBars = qb.History<TradeBar>(
    equity.Symbol,
    HistoryStartDate,
    HistoryEndDate,
    Resolution.Daily
)
.OrderBy(b => b.EndTime)
.ToList();

// =====================================
// Filter outlier DAILY bars (by totalDelta)
// =====================================

var dailyBars = new List<TradeBar>();
var filteredDailyOutliers = new List<TradeBar>();

foreach (var b in allDailyBars)
{
    var o = b.Open;
    var h = b.High;
    var l = b.Low;
    var c = b.Close;

    bool isOutlier = false;

    // Basic sanity: non-positive prices are junk
    if (o <= 0 || h <= 0 || l <= 0 || c <= 0)
    {
        isOutlier = true;
    }
    else
    {
        // totalDelta:
        //  - If the bar is green (close > open): % change low -> high
        //  - If the bar is red   (close < open): % change high -> low (negative)
        //  - If open == close: 0
        decimal pctTotalDelta = 0m;

        if (c > o)
        {
            // Green bar: (high - low) / low * 100
            if (l != 0)
            {
                pctTotalDelta = (h - l) / l * 100m;
            }
        }
        else if (c < o)
        {
            // Red bar: (low - high) / high * 100 (negative)
            if (h != 0)
            {
                pctTotalDelta = (l - h) / h * 100m;
            }
        }
        // else: doji-ish, pctTotalDelta stays 0m

        // Filter if |totalDelta| > 15%
        if (Math.Abs(pctTotalDelta) > TotalDeltaDailyThreshold)
        {
            isOutlier = true;
        }
    }

    if (isOutlier)
    {
        filteredDailyOutliers.Add(b);
    }
    else
    {
        dailyBars.Add(b);
    }
}

// =====================================
// Group into weekly bars
// =====================================

var weeklyBars = new List<TradeBar>();
var weeklyTradingDates = new Dictionary<DateTime, List<DateTime>>();
var filteredWeeklyOutliers = new List<TradeBar>();

// We'll store the date ranges for filtered WEEKLY outliers
var filteredWeeklyRanges = new List<(DateTime Start, DateTime End)>();

var groupedByWeek = dailyBars
    .GroupBy(bar =>
    {
        int dow = (int)bar.Time.DayOfWeek;
        int mondayOffset = dow == 0 ? -6 : 1 - dow;   // Sunday special case
        return bar.Time.Date.AddDays(mondayOffset);
    })
    .OrderBy(g => g.Key);

foreach (var group in groupedByWeek)
{
    var weekBars = group.OrderBy(x => x.Time).ToList();

    if (weekBars.Count < MinTradingDays)
        continue;

    var first = weekBars.First();
    var last  = weekBars.Last();

    var weekly = new TradeBar
    {
        Time   = first.Time,
        Symbol = first.Symbol,
        Open   = first.Open,
        High   = weekBars.Max(b => b.High),
        Low    = weekBars.Min(b => b.Low),
        Close  = last.Close,
        Volume = weekBars.Sum(b => b.Volume)
    };

    // =====================================
    // WEEKLY outlier detection
    // =====================================
    bool weeklyOutlier = false;

    if (weekly.Open > 0 && weekly.High / weekly.Open > 5m)
        weeklyOutlier = true;

    if (weekly.Low > 0 && weekly.Open / weekly.Low > 5m)
        weeklyOutlier = true;

    if (weeklyOutlier)
    {
        filteredWeeklyOutliers.Add(weekly);

        // Capture the start/end date range for this outlier week
        filteredWeeklyRanges.Add(
            (weekBars.First().Time.Date, weekBars.Last().Time.Date)
        );

        continue;   // skip this week
    }

    weeklyBars.Add(weekly);
    weeklyTradingDates[weekly.Time] =
        weekBars.Select(b => b.Time.Date).ToList();
}

// =====================================
// Build weekly JSON records
// =====================================

var weeklyRecords = new List<WeeklyBarRecord>();

foreach (var bar in weeklyBars)
{
    decimal open  = bar.Open;
    decimal high  = bar.High;
    decimal low   = bar.Low;
    decimal close = bar.Close;

    // Percentages
    decimal pctOpenToHigh  = open != 0 ? (high - open)  / open * 100m : 0m;
    decimal pctOpenToLow   = open != 0 ? (low  - open)  / open * 100m : 0m;
    decimal pctOpenToClose = open != 0 ? (close - open) / open * 100m : 0m;

    // totalDelta (weekly):
    //  - Green week (close > open): % change low -> high
    //  - Red week   (close < open): % change high -> low (negative)
    //  - Flat week  (open == close): 0
    decimal pctTotalDelta = 0m;

    if (close > open)
    {
        // Green week: (high - low) / low * 100
        if (low != 0)
        {
            pctTotalDelta = (high - low) / low * 100m;
        }
    }
    else if (close < open)
    {
        // Red week: (low - high) / high * 100 (negative)
        if (high != 0)
        {
            pctTotalDelta = (low - high) / high * 100m;
        }
    }
    // else: doji-ish week, keep 0m

    weeklyTradingDates.TryGetValue(bar.Time, out var dates);
    var endDate = dates?.Last() ?? bar.Time.Date;

    weeklyRecords.Add(new WeeklyBarRecord
    {
        startDate    = bar.Time.Date.ToString("yyyy-MM-dd"),
        endDate      = endDate.ToString("yyyy-MM-dd"),
        open         = Round2(open),
        high         = Round2(high),
        low          = Round2(low),
        close        = Round2(close),
        volume       = Round2(bar.Volume),
        tradingDays  = dates?.Count ?? 0,
        openToHigh   = Round2(pctOpenToHigh),
        openToLow    = Round2(pctOpenToLow),
        openToClose  = Round2(pctOpenToClose),
        totalDelta   = Round2(pctTotalDelta)
    });
}

// =====================================
// Compute statistics
// =====================================

var openToHighValues  = weeklyRecords.Select(r => r.openToHigh).ToList();
var openToLowValues   = weeklyRecords.Select(r => r.openToLow).ToList();
var openToCloseValues = weeklyRecords.Select(r => r.openToClose).ToList();
var totalDeltaValues  = weeklyRecords.Select(r => r.totalDelta).ToList();

Console.WriteLine("Weekly Percentage Move Statistics (2-decimal rounded):\n");

// openToHigh
Console.WriteLine(
    $"openToHigh  -> mean: {Round2(openToHighValues.Average())}%, " +
    $"median: {Round2(Median(openToHighValues))}%, " +
    $"std dev: {Round2(StdDev(openToHighValues))}%");

// openToLow
Console.WriteLine(
    $"openToLow   -> mean: {Round2(openToLowValues.Average())}%, " +
    $"median: {Round2(Median(openToLowValues))}%, " +
    $"std dev: {Round2(StdDev(openToLowValues))}%");

// openToClose
Console.WriteLine(
    $"openToClose -> mean: {Round2(openToCloseValues.Average())}%, " +
    $"median: {Round2(Median(openToCloseValues))}%, " +
    $"std dev: {Round2(StdDev(openToCloseValues))}%");

// totalDelta
Console.WriteLine(
    $"totalDelta  -> mean: {Round2(totalDeltaValues.Average())}%, " +
    $"median: {Round2(Median(totalDeltaValues))}%, " +
    $"std dev: {Round2(StdDev(totalDeltaValues))}%");

Console.WriteLine();

// =====================================
// Report filtered DAILY outliers
// =====================================

if (filteredDailyOutliers.Count > 0)
{
    Console.WriteLine("Filtered DAILY outlier bars:");
    foreach (var b in filteredDailyOutliers.OrderBy(x => x.Time))
    {
        Console.WriteLine(
            $"{b.Time:yyyy-MM-dd} | O={b.Open} H={b.High} L={b.Low} C={b.Close} V={b.Volume}"
        );
    }
}
else
{
    Console.WriteLine("No DAILY outlier bars were filtered.");
}

Console.WriteLine();

// =====================================
// Report filtered WEEKLY outliers
// =====================================

if (filteredWeeklyOutliers.Count > 0)
{
    Console.WriteLine("Filtered WEEKLY outlier bars:");
    foreach (var w in filteredWeeklyOutliers.OrderBy(x => x.Time))
    {
        Console.WriteLine(
            $"{w.Time:yyyy-MM-dd} | O={w.Open} H={w.High} L={w.Low} C={w.Close} V={w.Volume}"
        );
    }
}
else
{
    Console.WriteLine("No WEEKLY outlier bars were filtered.");
}

Console.WriteLine();

// =====================================
// Build JSON structures for filtered data
// =====================================

// Daily filtered bars -> full bar info
var filteredDayRecords = filteredDailyOutliers
    .OrderBy(b => b.Time)
    .Select(b => new
    {
        date   = b.Time.Date.ToString("yyyy-MM-dd"),
        open   = Round2(b.Open),
        high   = Round2(b.High),
        low    = Round2(b.Low),
        close  = Round2(b.Close),
        volume = Round2(b.Volume)
    })
    .ToList();

// Weekly filtered ranges -> just start/end dates
var filteredWeekRecords = filteredWeeklyRanges
    .Select(r => new
    {
        startDate = r.Start.ToString("yyyy-MM-dd"),
        endDate   = r.End.ToString("yyyy-MM-dd")
    })
    .ToList();

// =====================================
// Final JSON object (top-level)
// =====================================

var finalObject = new
{
    ticker = TickerSymbol,
    analysisStartDate = HistoryStartDate.ToString("yyyy-MM-dd"),
    analysisEndDate = HistoryEndDate.ToString("yyyy-MM-dd"),
    minTradingDayWeekThreshold = MinTradingDays,
    numWeeks = weeklyRecords.Count,

    // New properties placed BEFORE weeklyData
    totalDeltaDailyThreshold = TotalDeltaDailyThreshold,
    filteredDays = filteredDayRecords,
    filteredWeeks = filteredWeekRecords,

    weeklyData = weeklyRecords
};

// Output JSON
JsonConvert.SerializeObject(finalObject, Formatting.Indented)
